# CEO-Request Challenge

## 1. Summary of Problem Statement

❓ **How could Olist improve its profit** ❓

### P&L Rules

#### Revenues  
**Sales fees:** Olist takes a **10% cut** on the product price (excl. freight) of each order delivered  
**Subscription fees:** Olist charges **80 BRL by month** per seller

#### Costs
**Reputation costs** *estimated* per order with bad reviews (<= 3 stars)  

💡 In the long term, bad customer experience has business implications: low repeat rate, immediate customer support cost, refunds or unfavorable word of mouth communication. We will assume that we have an estimate measure of the monetary cost for each bad review:
```python
# review_score: cost(BRL)
{'1 star': 100
'2 stars': 50
'3 stars': 40
'4 stars': 0
'5 stars': 0}
```

**IT costs:** Olist's **total cumulated IT Costs** scale with the square root of the total number of sellers that has ever join the platform, as well as the square root of the total cumulated number of products that was ever sold.  

$IT\_costs = \alpha * \sqrt{n\_sellers} + \beta * \sqrt{n\_products}$  
Olist's data team gave us the following values for these scaling parameters:
- $\alpha = 3157.27$
- $\beta = 978.23$

💡 Both number of sellers to manage and sales transaction are costly for IT systems.  
💡 Yet square roots reflect scale-effects: IT-system are often more efficient as they grow bigger.  
💡 Alpha > Beta means that "Olist has a lower IT Cost with few sellers selling a lot of products rather than the opposite  
- with **1000 sellers** and a total of **100 products** sold, the total IT cost accumulates to 109,624 BRL
- with **100 sellers** and a total of **1000 products** sold, the total IT cost accumulates to 62,507 BRL

Finally, The IT department also told you that since the birth of the marketplace, cumulated IT costs have amounted to **500,000 BRL**.

### Key Findings, so far

- `wait_time` is the most significant factor behind low review scores  
- `wait_time` is made up of seller's `delay_to_carrier` + `carrier_delivery_time`.  
- The latter being outside of Olist's direct control, improving it is not a quick-win recommendation
- On the contrary, a better selection of `sellers` can positively impact the `delay_to_carrier` and reduce the number of bad `review_scores` on Olist.
- Comments of the bad reviews showed that some were linked to the seller or to the product itself.

💡 We recommend you to start with the the guided seller analysis in part 2 below  
💪 But feel free to investigate into other hypothesis instead with part 3

## 2. Should Olist remove under-performing sellers from its marketplace? 🕵🏻
*(recommended)*

To analyze the impact of removing the worse sellers from Olist's marketplace, we will perform a **what-if analysis**

👉 **What would have happened if Olist had never accepted these sellers in the first place?**  

*(In practice, it's hard to know in advance who is a good seller, but let's start with this approach and iterate later).*

### 2.1 Data Preparation

Compute, for each `seller_id`, and cumulated since the beginning:
- the `revenues` it brings
- the `review_costs` associated with all its bad reviews
- the resulting `profits` (revenues - costs)

👉 Write down a step-by-step strategy to create the DataFrame you need


⚠️ Don't start from scratch, update your existing package 😉

Starting from the `Seller` class of your `olist` package:

Edit the `get_training_data` method so that the DataFrame it returns contains the fields:
- `revenues`: sum of subscription and sales fees revenues
- `cost_of_reviews`: sum of costs associated with bad reviews
    - edit the `get_review_score` method of your `Seller` class as an intermediary step
- `profits`: `revenues` - `cost_of_reviews`

### 2.2 What-if Analysis

👉 Time to perform the actual analysis, here are our steps:  

1️⃣ Create a method that will help us update the IT Costs after removing sellers along with the products they sold

2️⃣ Sort sellers by increasing profits

3️⃣ Remove sellers one by one, starting from the one with the lowest profit.
- For each number of sellers to remove, compute the financial impact on Olist global profits.

4️⃣ Find an optimal number of sellers to remove that maximizes either Olist's profit margin or profit.

## 3. Investigate other Approaches 🕵️
*(optional)*

- Should Olist remove the worst performing products / categories from its marketplace entirely?
- Should Olist remove only consistently underperforming sellers, after it has a honeymoon period of few months?
- Should Olist restrict seller/customer pairs between certain states to avoid delays?
- Should Olist acquire new sellers, with some cost assumptions to be suggested?
- ...

## Your turn!

🧺 Keep this notebook tidy! 

📅 🗣 📊 You will present your insights to your favorite TA at the end of this `Communicate` unit.

In [11]:
import sys
sys.path.append('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/olist')

from seller import Seller

In [6]:
# instance
seller_instance = Seller()

# données
training_data = seller_instance.get_training_data()
training_data.head()

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,share_of_one_stars,share_of_five_stars,review_score,cost_of_reviews,n_orders,quantity,quantity_per_order,sales,revenues,profits
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,0.333333,0.333333,3.00,140,3,3,1.000,218.70,341.870,201.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,0.050000,0.725000,4.55,240,40,41,1.025,11703.07,2290.307,2050.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,0.000000,1.000000,5.00,0,1,1,1.000,158.00,15.800,15.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,0.000000,1.000000,5.00,0,1,1,1.000,79.99,7.999,7.999
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1.000000,0.000000,1.00,100,1,1,1.000,167.99,16.799,-83.201


In [7]:
import pandas as pd


# Calcul des revenus totaux
total_revenues = training_data['revenues'].sum()
print(f"Revenus totaux : {total_revenues}")

# Calcul des profits totaux
total_profits = training_data['profits'].sum()
print(f"Profits totaux : {total_profits}")

# Calcul du coût total des avis associés aux mauvaises critiques
total_cost_of_reviews = training_data['cost_of_reviews'].sum()
print(f"Coût total des avis associés aux mauvaises critiques : {total_cost_of_reviews}")

Revenus totaux : 2785818.444
Profits totaux : 1167608.4440000001
Coût total des avis associés aux mauvaises critiques : 1618210


## Create a method that will help us update the IT Costs after removing sellers along with the products they sold

In [8]:
import sys
sys.path.append('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/olist')

from seller import Seller

seller_instance = Seller()

new_it_costs = seller_instance.update_it_costs(n_sellers=90, n_products=900)
print(f"Nouveaux coûts informatiques après suppression : {new_it_costs}")

Nouveaux coûts informatiques après suppression : 59299.393164359455


## Sort sellers by increasing profits

In [10]:
sorted_sellers = training_data.sort_values(by='profits')
print(sorted_sellers.head(10))

                             seller_id            seller_city seller_state  \
769   6560211a19b47992c3666cc44a7e94c0              sao paulo           SP   
2358  4a3ca9315b744ce9f8e9374361493884               ibitinga           SP   
1357  cc419e0650a3c5ba77189a1882b7556a            santo andre           SP   
945   ea8482cd71df3c1969d7b9473ff13abc              sao paulo           SP   
315   8b321bb669392f5163d04c59e235e066              sao paulo           SP   
453   1f50f920176fa81dab994f9023523100  sao jose do rio preto           SP   
1213  d2374cbcbb3ca4ab1086534108cc3ab7               ibitinga           SP   
1132  7c67e1448b00f6e969d365cea6b010ab        itaquaquecetuba           SP   
2023  cca3071e3e9bb7d12640c9fbe2301306               ibitinga           SP   
2687  1835b56ce799e6a4dc4eddc053f04066               ibitinga           SP   

      delay_to_carrier  wait_time     date_first_sale      date_last_sale  \
769                0.0   9.539486 2017-02-17 07:45:11 2018-08-29

## Remove sellers one by one, starting from the one with the lowest profit. For each number of sellers to remove, compute the financial impact on Olist global profits.

In [13]:
results = []
initial_global_profit = training_data['profits'].sum() # Calcul du profit global initial

for i in range(len(sorted_sellers)):
    # Supprime les vendeurs avec les profit les plus bas
    remaining_sellers = sorted_sellers.iloc[i+1:]

    # Recalcule le profit global après le retrait du vendeur
    updated_global_profit = remaining_sellers['profits'].sum()

    # Stocker dans la lite
    results.append({
        'n_sellers_removed': i + 1,
        'updated_global_profit': updated_global_profit,
        'profit_change': updated_global_profit - initial_global_profit
    })

results_df = pd.DataFrame(results)
print(results_df.head(20))

    n_sellers_removed  updated_global_profit  profit_change
0                   1            1189127.961      21519.517
1                   2            1206880.669      39272.225
2                   3            1224461.827      56853.383
3                   4            1240004.075      72395.631
4                   5            1252890.506      85282.062
5                   6            1262816.585      95208.141
6                   7            1271834.193     104225.749
7                   8            1279851.804     112243.360
8                   9            1287230.815     119622.371
9                  10            1294366.394     126757.950
10                 11            1299783.533     132175.089
11                 12            1304995.280     137386.836
12                 13            1310131.123     142522.679
13                 14            1315105.420     147496.976
14                 15            1319663.028     152054.584
15                 16            1324102

## Find an optimal number of sellers to remove that maximizes either Olist's profit margin or profit.

In [14]:
# Trouve la ligne où les profits globaux sont max
optimal_row = results_df.loc[results_df['updated_global_profit'].idxmax()]

# Extrait info pertinentes
optimal_sellers_to_remove = optimal_row['n_sellers_removed']
optimal_global_profit = optimal_row['updated_global_profit']
print(f"Nombre optimal de vendeurs à retirer : {optimal_sellers_to_remove}")
print(f"Profit global après le retrait : {optimal_global_profit}")

Nombre optimal de vendeurs à retirer : 376.0
Profit global après le retrait : 1474071.063


# Optional

## Should Olist remove the worst performing products / categories from its marketplace entirely?

## Should Olist remove only consistently underperforming sellers, after it has a honeymoon period of few months?

## Should Olist restrict seller/customer pairs between certain states to avoid delays?

## Should Olist acquire new sellers, with some cost assumptions to be suggested?